## Install geemap

In [ ]:
#!pip install geemap

In [ ]:
#!earthengine task cancel all
#!earthengine task list

In [6]:
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nXHGQHe3i2CvuQTBiB_5sffph012csz4VS6AuhI0cMU&tc=bNTIdofe1u_shnU2tJ3gvNIssuuhhSuKd_PkqoiBHEw&cc=o8qcRTxWbrIcm_4-6u-fVL-oJ5_vIaSpAplW61BOJwE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqCN2HKhybIAWDv7mOWmNMrjMaiFwK520KR1u-Gi4dXj7P3qk3pUv0

Successfully saved authorization token.


In [ ]:
#import geemap
#Map = geemap.Map()
#Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Extract Precipitation

### Constants:

In [10]:
# Ref: https:#developers.google.com/earth-engine/tutorials/community/ph-ug-temp

'Municipalities:'
# Import country boundaries feature collection.
dataset = ee.FeatureCollection('projects/ee-dsrestrepo/assets/mpio')
#Codes = dataset.aggregate_array('MPIOS')
Codes = {
    'Medellín':'05001',
    'Itagüí': '5360',
    'Barranquilla': '8001',
}

'Dates:'
# Define a date range of interest; here, a start date is defined and the end
start = ee.Date('2007-01-01')
dateRange = ee.DateRange(start, start.advance(13, 'year'))

'Satellite data:'
modis = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
# Filter the precipitation collection to include only images intersecting the desired date range.
mod11a2 = modis.filterDate(dateRange)
# Select only the precipitation data band.
modLSTday = mod11a2.select('precipitation')
# Set image acquisition time.
def func_opl(img):
  return img.copyProperties(img, ['system:time_start'])

modLSTc = modLSTday.map(func_opl)

### Extract precipitation for each municipality:

In [11]:
def func_hfv (image):
  date = image.date().format('yyyy-MM-dd')
  value = image.reduceRegion(
    reducer= ee.Reducer.mean(), 
    geometry= Municipality.geometry(), 
    scale= 1000).get('precipitation')
  return ee.Feature(None, {'value': value, 'date': date})

# Get time series data:
def get_time_series(Municipality):
  timeSeries = modLSTc.map(func_hfv)
  return timeSeries

In [12]:
for mun in Codes:
  code = Codes[mun]
  print(code)
  # Apply filter where municipalty code equals code.
  Municipality = dataset.filter(ee.Filter.eq('MPIOS', code))
  timeSeries = get_time_series(Municipality)

  task = ee.batch.Export.table.toDrive(** {
    'collection': timeSeries,
    'folder': 'GEE_precipitation_TOP10',
    'description': 'precipitation_' + code,
    'fileFormat': 'CSV',
    'selectors': ['value', 'date']
  })
  
  task.start()

print("Done!")

05001
Done!
